### Quantized version of the ML model
(Non Quantized version of this notebook is done by Shreyas, credit should go to him for the non Quantized model)

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

from load_data import load_dance_data
from load_data import one_hot_encoder
from sklearn.utils import shuffle

import numpy as np
import os

In [2]:
dataset = os.getcwd() + '\Dance_data'
sampling_rate = 10
window_size = 2.56

X, y = load_dance_data(dataset, sampling_rate, window_size)



Files read from  Hair.txt
Files read from  Rocket.txt
Files read from  Zig_Zag.txt


In [3]:
#%%capture cap --no-stderr

hair_train = []
rocket_train = []
zigzag_train = []
#print(len(X))
for i,j in enumerate(y):
    if j == [0]:
        hair_train.append(X[i])
for i,j in enumerate(y):
    if j == [1]:
        rocket_train.append(X[i])
for i,j in enumerate(y):
    if j == [2]:
       zigzag_train.append(X[i])
    


### Print  KNN training set

# Training of model by pytorch

In [4]:
"""
hair_train_int = np.asarray(hair_train, dtype = np.int)
rocket_train_int = np.asarray(rocket_train, dtype = np.int)
zigzag_train_int = np.asarray(zigzag_train, dtype = np.int)
print("int hair_train[", len(hair_train_int) ,"][40]",' = {' , sep='', end='')
for i,a in enumerate(hair_train_int):
    if i == len(hair_train_int) - 1:
        print('{',' ,'.join(map(str, a)),"}};")
    else:
        print('{',' ,'.join(map(str, a)), '},')

print()
print()
print()
print("int rocket_train_int[", len(rocket_train_int) ,"][40]",' = {' , sep='', end='')
for i,a in enumerate(rocket_train_int):
    if i == len(rocket_train_int) - 1:
        print('{',' ,'.join(map(str, a)),"}};")
    else:
        print('{',' ,'.join(map(str, a)), '},')
print()
print()
print()
print("int zigzag_train_int[", len(zigzag_train_int) ,"][40]",' = {' , sep='', end='')
for i,a in enumerate(zigzag_train_int):
    if i == len(zigzag_train_int) - 1:
        print('{',' ,'.join(map(str, a)),"}};")
    else:
        print('{',' ,'.join(map(str, a)), '},')
"""

'\nhair_train_int = np.asarray(hair_train, dtype = np.int)\nrocket_train_int = np.asarray(rocket_train, dtype = np.int)\nzigzag_train_int = np.asarray(zigzag_train, dtype = np.int)\nprint("int hair_train[", len(hair_train_int) ,"][40]",\' = {\' , sep=\'\', end=\'\')\nfor i,a in enumerate(hair_train_int):\n    if i == len(hair_train_int) - 1:\n        print(\'{\',\' ,\'.join(map(str, a)),"}};")\n    else:\n        print(\'{\',\' ,\'.join(map(str, a)), \'},\')\n\nprint()\nprint()\nprint()\nprint("int rocket_train_int[", len(rocket_train_int) ,"][40]",\' = {\' , sep=\'\', end=\'\')\nfor i,a in enumerate(rocket_train_int):\n    if i == len(rocket_train_int) - 1:\n        print(\'{\',\' ,\'.join(map(str, a)),"}};")\n    else:\n        print(\'{\',\' ,\'.join(map(str, a)), \'},\')\nprint()\nprint()\nprint()\nprint("int zigzag_train_int[", len(zigzag_train_int) ,"][40]",\' = {\' , sep=\'\', end=\'\')\nfor i,a in enumerate(zigzag_train_int):\n    if i == len(zigzag_train_int) - 1:\n        pri

In [5]:
X, y = shuffle(X, y)
X_train, X_test = X[:int(len(X)*0.8), :], X[int(len(X)*0.8):, :]
y_train, y_test = y[:int(len(X)*0.8), :], y[int(len(X)*0.8):, :]

X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float()
y_train = y_train.reshape(y_train.shape[0]).long()

X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float()
y_test = y_test.reshape(y_test.shape[0]).long()

input_size = X.shape[1]
output_size = one_hot_encoder(y).shape[1]

In [6]:
hidden_states = [256, 128]
from torch.quantization import QuantStub, DeQuantStub

model = nn.Sequential(QuantStub(),
            nn.Linear(input_size, hidden_states[0]),
            nn.ReLU(),
            nn.Linear(hidden_states[0], hidden_states[1]),
            nn.ReLU(),
            nn.Linear(hidden_states[1], output_size),
            nn.Sigmoid(),
            DeQuantStub())

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [7]:
# Prepare data for training and testing
import torch.utils.data

class Prepare_data(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        x = self.inputs[index]
        y = self.labels[index]
        return x, y


In [8]:
trainloader = torch.utils.data.DataLoader(Prepare_data(X_train, y_train), batch_size=32, shuffle=True)

testloader = torch.utils.data.DataLoader(Prepare_data(X_test, y_test), batch_size=32)

In [9]:
epochs = 15

# Define the loss
criterion = nn.NLLLoss()
# Optimizers require the parameters to optimize and a learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=0.003)

for e in range(epochs):
    running_loss = 0
    for inputs, labels in trainloader:
    
        # Training pass
        optimizer.zero_grad()
        
        output = model(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(trainloader)}")

Training loss: -0.8277832199545467
Training loss: -0.9878954571836135
Training loss: -0.9908285912345437
Training loss: -0.9919883714002722
Training loss: -0.9927490248399622
Training loss: -0.9933833655189065
Training loss: -0.9938541510525871
Training loss: -0.9941380374571857
Training loss: -0.9945162920390859
Training loss: -0.9948355169857249
Training loss: -0.9950437160099254
Training loss: -0.9949472301146564
Training loss: -0.9952743474174949
Training loss: -0.9950617201188031
Training loss: -0.995787006967208


In [10]:
y_pred = model(X_test)
y_pred = y_pred.cpu()
result = y_pred.data.numpy()
array_res = np.reshape(result, (1, -1))
#array_res

In [11]:
model.eval()
test_acc = 0.0
count = 0
for i, (inputs, labels) in enumerate(testloader):
    # Predict classes using images from the test set
    outputs = model(inputs)
    _, prediction = torch.max(outputs.data, 1)

    test_acc += torch.sum(prediction == labels.data).float()
    count += 1

# Compute the average acc and loss over all 10000 test images
test_acc = test_acc / count
int(test_acc)

14

In [2]:
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.handlers import EarlyStopping

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def score_function(engine):
    val_loss = engine.state.metrics['crossentropy']
    accuracy = engine.state.metrics['accuracy']
    return accuracy #-val_loss

trainer = create_supervised_trainer(model, optimizer, loss)
evaluator = create_supervised_evaluator(model,
                                        metrics={
                                            'accuracy': Accuracy(),
                                            'crossentropy': Loss(loss)
                                            })

handler = EarlyStopping(patience=5, score_function=score_function, trainer=trainer)
# Note: the handler is attached to an *Evaluator* (runs one epoch on validation dataset).
evaluator.add_event_handler(Events.COMPLETED, handler)

@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    evaluator.run(trainloader)
    metrics = evaluator.state.metrics
    print("Training Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}"
          .format(trainer.state.epoch, metrics['accuracy'], metrics['crossentropy']))

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    evaluator.run(testloader)
    metrics = evaluator.state.metrics
    print("Validation Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}"
          .format(trainer.state.epoch, metrics['accuracy'], metrics['crossentropy']))

trainer.run(trainloader, max_epochs=35)

NameError: name 'nn' is not defined

# Quantization of trained model

## Float inference using inference algorithm

Matrix multiplicaition of weights and inputs matrices to get output for linear layers. The float version is compared to the int version of validation and comparison of accuracy, and for debugging purposes as well.

For Relu, we simply set the output to zero if it is less than 0; else we do not change it (Hence the name rectified linear)

In [1]:
import sys, os
import torch
save_path = os.path.abspath(os.getcwd() + "/../../sw1_ml/src/model/")
print((save_path))
sys.path.insert(1, save_path)
from neural_network_model import neural_network_model_1

model = neural_network_model_1(40, [256, 128], 3)
model.load_state_dict(torch.load("NN_v1_3moves.pt"))
#first_model.eval()

print(model)

G:\CG4002\CG4002_B18\sw1_ml\src\data


ModuleNotFoundError: No module named 'neural_network_model'

In [14]:

torch.set_printoptions(edgeitems=10000)
np.set_printoptions(edgeitems=10000)

layer_0_float = model[1].weight.data
layer_2_flaot = model[3].weight.data
layer_4_flaot = model[5].weight.data


'\ntorch.set_printoptions(edgeitems=10000)\nnp.set_printoptions(edgeitems=10000)\n\nlayer_0_float = model[1].weight.data\nlayer_2_flaot = model[3].weight.data\nlayer_4_flaot = model[5].weight.data\n\n'

In [15]:
classes = ['Hair', 'Rocket', 'Zig-Zag']
testloader = torch.utils.data.DataLoader(Prepare_data(X_test, y_test), batch_size=20)
dataiter = iter(testloader)
inputs, labels = dataiter.next()

print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(len(labels))), '\n')

outputs = model(inputs)

_, predicted = torch.max(outputs, 1)

print('Predicted:   ', ' '.join('%5s' % classes[predicted[j]] for j in range(len(labels))))

GroundTruth:   Hair Zig-Zag  Hair  Hair Zig-Zag  Hair  Hair Zig-Zag Zig-Zag  Hair  Hair Rocket  Hair Zig-Zag Zig-Zag Zig-Zag  Hair Rocket Zig-Zag Rocket 

Predicted:     Hair Zig-Zag  Hair  Hair Zig-Zag  Hair  Hair Zig-Zag Zig-Zag  Hair  Hair Zig-Zag  Hair Zig-Zag Zig-Zag Zig-Zag  Hair Zig-Zag Zig-Zag Rocket


In [16]:
import time
start_time = time.time()

inp = inputs.numpy()
#print(inp)
layer_0_out_float = []

#print (np.array2string(layer_4_flaot.numpy(), separator=', '))

for row in layer_0_float:
    sum = 0
    for col in range(len(row)):
        sum += row[col] * inp[0][col] 
    layer_0_out_float.append(sum)
#layer_1_out = np.cross(layer_1_int, inp)
#print(layer_0_out)
layer_0_out_float = np.asarray(layer_0_out_float)
#print(layer_0_out_float)

#relu
for i in range(len(layer_0_out_float)):
    if layer_0_out_float[i] < 0:
        layer_0_out_float[i]  = 0
    else:
        layer_0_out_float[i]  = layer_0_out_float[i]
#print(layer_0_out_float)

NameError: name 'layer_0_float' is not defined

In [ ]:
layer_2_out_float = []
#print(layer_0_float)
for row in layer_2_flaot:
    sum = 0
    for col in range(len(row)):
        sum += row[col] * layer_0_out_float[col] 
    layer_2_out_float.append(sum)
#layer_1_out = np.cross(layer_1_int, inp)
#print(layer_0_out)
layer_2_out_float = np.asarray(layer_2_out_float)
#print(layer_0_out)

#relu
for i in range(len(layer_2_out_float)):
    if layer_2_out_float[i] < 0:
        layer_2_out_float[i]  = 0
    else:
        layer_2_out_float[i]  = layer_2_out_float[i]
#print(layer_2_out_float)

In [ ]:
layer_4_out_float = []
#print(layer_0_float)
for row in layer_4_flaot:
    sum = 0
    for col in range(len(row)):
        sum += row[col] * layer_2_out_float[col] 
    layer_4_out_float.append(sum)
#layer_1_out = np.cross(layer_1_int, inp)
#print(layer_0_out)
layer_4_out_float = np.asarray(layer_4_out_float)
#print(layer_0_out)

#relu
for i in range(len(layer_4_out_float)):
    if layer_4_out_float[i] < 0:
        layer_4_out_float[i]  = 0
    else:
        layer_4_out_float[i]  = layer_4_out_float[i]
#print(layer_4_out_float)

In [ ]:
tensor = torch.Tensor([layer_4_out_float])
m =  nn.Sigmoid()
out = m(tensor)
_, pred = torch.max(out, 1)
print("Time taken" ,(time.time() - start_time) * 1000 , 'ms')
print(pred)
print('Predicted:   ', ' '.join('%5s' % classes[pred[0]]))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(len(labels))))

## INT INFERENCE
### Post-training static quantization
Quantisation steps:

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import os
import time
import sys
import torch.quantization

model.qconfig = torch.quantization.get_default_qconfig('fbgemm')
torch.quantization.prepare(model, inplace=True) 

evaluator = create_supervised_evaluator(model,
                                        metrics={
                                            'accuracy': Accuracy(),
                                            'crossentropy': Loss(loss)
                                            })
evaluator.add_event_handler(Events.COMPLETED, handler)
testloader = torch.utils.data.DataLoader(Prepare_data(X_test, y_test), batch_size=32)
evaluator.run(testloader)

torch.quantization.convert(model, inplace=True)

layer_0_int = (model[1].weight().int_repr().numpy())
layer_2_int = (model[3].weight().int_repr().numpy())
layer_4_int = (model[5].weight().int_repr().numpy())

layer_0_scale = model[1].scale
layer_2_scale = model[3].scale
layer_4_scale = model[5].scale

layer_0_zero = model[1].zero_point
layer_2_zero = model[3].zero_point
layer_4_zero = model[5].zero_point

inp_scale = model[0].scale
inp_zero = model[0].zero_point


#print(','.join([str(x) for x in inp[0]]))
#print((layer_0_int[0])/ layer_0_scale )

In [ ]:
inp = inputs.numpy().astype(int)
print(inp)
start_time = time.time()
#print(layer_0_int)
layer_0_out = []
for row in layer_0_int:
    sum = 0
    for col in range(len(row)):
        sum += row[col] * inp[0][col]
    layer_0_out.append(int(sum / inp_scale / layer_0_scale ) )
#layer_1_out = np.cross(layer_1_int, inp)
#print(layer_0_out)
#layer_0_out = np.asarray(layer_0_out)
#print(layer_0_out)

#relu
for i in range(len(layer_0_out)):
    if layer_0_out[i] < 0:
        layer_0_out[i]  = 0
    else:
        layer_0_out[i]  = layer_0_out[i] 
layer_0_out = np.asarray(layer_0_out )
#print(layer_0_out)

In [ ]:
layer_2_in = layer_0_out / layer_0_scale
layer_2_out = []
for row in layer_2_int:
    sum = 0
    for col in range(len(row)):
        sum += row[col] * layer_2_in[col]
    layer_2_out.append(int(sum / layer_2_scale))
#print(layer_2_out)


for i in range(len(layer_2_out)):
    if layer_2_out[i] < 0:
        layer_2_out[i]  = 0

layer_2_out = np.asarray(layer_2_out)
#print(layer_2_out)


In [ ]:
layer_4_in = layer_2_out / layer_2_scale
layer_4_out = []
for row in layer_4_int:
    sum = 0
    for col in range(len(row)):
        sum += row[col] *  layer_4_in[col]
    layer_4_out.append(int(sum) / layer_4_scale)       

#print(layer_4_scale)    
layer_4_out = np.asarray(layer_4_out)
#print(layer_4_out)


In [ ]:
tensor = torch.Tensor([layer_4_out])
m =  nn.Sigmoid()
out = m(tensor)
_, pred = torch.max(out, 1)
print("Time taken" ,(time.time() - start_time) * 1000 , 'ms')
print(pred)
print('Predicted:   ', ' '.join('%5s' % classes[pred[0]]))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(len(labels))))

print(layer_0_float.shape)


# KNN Inference

In [ ]:
inputs_knn = inputs.numpy()[0]
start_time = time.time()
hair_dist_min = 10e9
rocket_dist_min = 10e9
zigzag_dist_min = 10e9

for i in range(128):
    hair_dist = 0
    rocket_dist = 0
    zigzag_dist = 0
    for j in range(40):
        hair_dist += abs(hair_train[i][j] - inputs_knn[j])
        rocket_dist += abs(rocket_train[i][j] - inputs_knn[j])
        zigzag_dist += abs(zigzag_train[i][j] - inputs_knn[j])
    if hair_dist < hair_dist_min:
        hair_dist_min = hair_dist
    if rocket_dist < rocket_dist_min:
        rocket_dist_min = rocket_dist
    if zigzag_dist < zigzag_dist_min:
        zigzag_dist_min = zigzag_dist
print("Time taken" ,(time.time() - start_time) * 1000 , 'ms')        
print(hair_dist_min, rocket_dist_min,zigzag_dist_min)

#  Dump weights to C++ int array 

In [3]:
%%capture cap --no-stderr

print("int inputs[40]={", end = '')
for i in range(len(inp[0])):
    if i < len(inp[0]) - 1:
        print(inp[0][i], end=',')
    else:
        print(inp[0][i], end='};')
        print()

print("int inpScale = " , inp_scale[0].numpy() , ';')    
print("int scaleMat0 = " , layer_0_scale , ';')    
print("int scaleMat1 = " , layer_2_scale , ';')    
print("int scaleMat2 = " , layer_4_scale , ';')    

print("int linearMat0[256][40] = {", end = '')
for i in layer_0_int:
    print('{',end = '')
    for j in range(len(i)):
        if j < (len(i) - 1):
            print(i[j], end = ', ')
        else:
            print(i[j], end = '')
    print('},')

print()
print()
print()
print("int linearMat1[128][256] = {")
for i in layer_2_int:
    print('{',end = '')
    for j in range(len(i)):
        if j < (len(i) - 1):
            print(i[j], end = ', ')
        else:
            print(i[j], end = '')
    print('},')
print()
print()
print()
    
print("int linearMat2[3][128] = {")
for i in layer_4_int:
    print('{',end = '')
    for j in range(len(i)):
        if j < (len(i) - 1):
            print(i[j], end = ', ')
        else:
            print(i[j], end = '')
    print('},')

with open('cppWeights.txt', 'w') as f:
    f.write(cap.stdout)

NameError: name 'inp' is not defined

In [ ]:
with open('cppWeights.txt', 'w') as f:
    f.write(cap.stdout)